In [1]:
import os
import json
import pandas as pd
import traceback


In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain.chat_models import ChatOpenAI

In [7]:
key = os.getenv('OPEN_API_KEY')

In [11]:
llm= ChatOpenAI(openai_api_key=key, model_name='gpt-3.5-turbo',temperature=0.5)

In [12]:
from langchain.llms import OpenAI
from langchain.prompts import PipelinePromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2